In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import sklearn


books = pd.read_csv('Books.csv')
ratings = pd.read_csv('Ratings.csv')
users = pd.read_csv('Users.csv')

# Lets store users who had at least rated more than 200 books
x = ratings['User-ID'].value_counts() > 3

x[x].shape

y= x[x].index

ratings = ratings[ratings['User-ID'].isin(y)]

# Now join ratings with books

ratings_with_books = ratings.merge(books, on='ISBN')

number_rating = ratings_with_books.groupby('BOOK-TITLE')['Ratings'].count().reset_index()

number_rating.rename(columns={'Ratings':'num_of_rating'},inplace=True)

number_rating.head()

final_rating = ratings_with_books.merge(number_rating, on='BOOK-TITLE')

# Lets take those books which got at least 2 rating of user

final_rating = final_rating[final_rating['num_of_rating'] >= 2]


# lets drop the duplicates
final_rating.drop_duplicates(['User-ID','BOOK-TITLE'],inplace=True)

# Lets create a pivot table
book_pivot = final_rating.pivot_table(columns='User-ID', index='BOOK-TITLE', values= 'Ratings')

book_pivot.fillna(0, inplace=True)


from scipy.sparse import csr_matrix


book_sparse = csr_matrix(book_pivot)

# Now import our clustering algoritm which is Nearest Neighbors this is an unsupervised ml algo
from sklearn.neighbors import NearestNeighbors
model = NearestNeighbors(algorithm= 'brute')

model.fit(book_sparse)

distance, suggestion = model.kneighbors(book_pivot.iloc[4,:].values.reshape(1,-1), n_neighbors=2 )


distance




for i in range(len(suggestion)):
    print(book_pivot.index[suggestion[i]])
    

#keeping books name
book_names = book_pivot.index

#Recommend a book
def recommend_book(book_name):
    book_id = np.where(book_pivot.index == book_name)[0][0]
    distance, suggestion = model.kneighbors(book_pivot.iloc[book_id,:].values.reshape(1,-1), n_neighbors=2)
    
    for i in range(len(suggestion)):
            books = book_pivot.index[suggestion[i]]
            for j in books:
                if j == book_name:
                    print(f"You searched '{book_name}'\n")
                    print("The suggestion books are: \n")
                else:
                    print(j)

book_name = "The silence of the lambs"
recommend_book(book_name)
